# Import raw data files

In [1]:
import os

In [2]:
import pandas as pd

In [3]:
import numpy as np

In [4]:
import glob

In [ ]:
#path=r"C:\Users\yunan\Downloads\York U\Machine Learning Cert\ML1010\aclImdb"


In [6]:
#read train pos files
text=[]
file_names=[]
is_positive=[]
#path=r"C:\Users\yunan\Downloads\York U\Machine Learning Cert\ML1010\aclImdb\train\neg"
#filenames=glob.glob(path + "/*.txt")
with os.scandir(r"C:\Users\yunan\Downloads\York U\Machine Learning Cert\ML1010\Group project 1\aclImdb\train\pos") as filenames:
    for file in filenames:
        head, tail = os.path.split(file)
        file_names.append(tail)
        is_positive.append(1)
        with open(file, 'rb') as open_file:
            text.append(open_file.readlines()[0])

In [7]:
train_df=pd.DataFrame({'file':file_names, 'text':text, 'is_positive':is_positive})

In [ ]:
train_df

In [8]:
#read train neg files
with os.scandir(r"C:\Users\yunan\Downloads\York U\Machine Learning Cert\ML1010\Group project 1\aclImdb\train\neg") as filenames:
    for file in filenames:
        head,tail=os.path.split(file)
        file_names.append(tail)
        is_positive.append(0)
        with open(file,'rb') as open_file:
            text.append(open_file.readlines()[0])

In [22]:
train_df=pd.DataFrame({'file':file_names, 'text':text, 'is_positive':is_positive})

In [23]:
train_df['text']=train_df['text'].astype(str)

In [24]:
# reduce dataset size - select only 5000 records
positive_reviews = train_df[:2500]   #take first 2500 reviews which are positive
negative_reviews = train_df[22500:]  #last last 2500 reviews which are negative
new_train = positive_reviews.append(negative_reviews, ignore_index=True)

In [25]:
new_train

,file,text,is_positive
0,0_9.txt,b'Bromwell High is a cartoon comedy. It ran at...,1
1,10000_8.txt,b'Homelessness (or Houselessness as George Car...,1
2,10001_10.txt,b'Brilliant over-acting by Lesley Ann Warren. ...,1
3,10002_7.txt,b'This is easily the most underrated film inn ...,1
4,10003_8.txt,b'This is not the typical Mel Brooks film. It ...,1
...,...,...,...
4995,9998_4.txt,"b""Towards the end of the movie, I felt it was ...",0
4996,9999_3.txt,b'This is the kind of movie that my enemies co...,0
4997,999_3.txt,"b""I saw 'Descent' last night at the Stockholm ...",0
4998,99_1.txt,"b""Some films that you pick up for a pound turn...",0


#  Text Pre-processing

In [14]:
import re
import nltk
import matplotlib.pyplot as plt
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\yunan\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
from tokenize import tokenize

In [ ]:
wpt=nltk.WordPunctTokenizer()
stop_words=nltk.corpus.stopwords.words('english')

In [15]:
from nltk.tokenize.casual import casual_tokenize

In [17]:
from nltk.stem.snowball import SnowballStemmer #  Snowball stemmer > Porter stemmer
from nltk.tokenize.casual import casual_tokenize  # we use casual tokenize because this is colloquial text
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS as sklearn_stop_words  # sklearn stop words is larger than nltk

In [26]:
stemmer = SnowballStemmer("english")
def decontracted(phrase):
    # Taken from https://stackoverflow.com/questions/19790188/expanding-english-language-contractions-in-python
    
    # specific
    phrase = re.sub(r"won\'t", "will not", phrase)
    phrase = re.sub(r"can\'t", "can not", phrase)

    # general
    phrase = re.sub(r"n\'t", " not", phrase)
    phrase = re.sub(r"\'re", " are", phrase)
    phrase = re.sub(r"\'s", " is", phrase)
    phrase = re.sub(r"\'d", " would", phrase)
    phrase = re.sub(r"\'ll", " will", phrase)
    phrase = re.sub(r"\'t", " not", phrase)
    phrase = re.sub(r"\'ve", " have", phrase)
    phrase = re.sub(r"\'m", " am", phrase)
    return phrase


def tokenize_phrase(text):
    phrase = decontracted(text.replace('\\', '').replace("b'", "")) # remove backslashes and replace contractions
    tokens = casual_tokenize(phrase, reduce_len=True, strip_handles=True)
    normalized_tokens = [x.lower() for x in tokens] #  convert to all lowercase
    filtered_tokens = [x for x in normalized_tokens if x not in sklearn_stop_words] #  filter stop words
    filtered_tokens = [x for x in filtered_tokens if x and x not in '- \t\n."\':[...][\\]()/[br]<>*~,;!?'] #  filter punctuations
    stemmed_tokens = [stemmer.stem(w) for w in filtered_tokens] # perform stemming
    return stemmed_tokens


In [27]:
new_train['tokens'] = new_train['text'].apply(tokenize_phrase)
new_train

,file,text,is_positive,tokens
0,0_9.txt,b'Bromwell High is a cartoon comedy. It ran at...,1,"[bromwel, high, cartoon, comedi, ran, time, pr..."
1,10000_8.txt,b'Homelessness (or Houselessness as George Car...,1,"[homeless, houseless, georg, carlin, state, is..."
2,10001_10.txt,b'Brilliant over-acting by Lesley Ann Warren. ...,1,"[brilliant, over-act, lesley, ann, warren, bes..."
3,10002_7.txt,b'This is easily the most underrated film inn ...,1,"[easili, underr, film, inn, brook, cannon, sur..."
4,10003_8.txt,b'This is not the typical Mel Brooks film. It ...,1,"[typic, mel, brook, film, slapstick, movi, act..."
...,...,...,...,...
4995,9998_4.txt,"b""Towards the end of the movie, I felt it was ...",0,"[end, movi, felt, technic, felt, like, classro..."
4996,9999_3.txt,b'This is the kind of movie that my enemies co...,0,"[kind, movi, enemi, content, watch, time, bloo..."
4997,999_3.txt,"b""I saw 'Descent' last night at the Stockholm ...",0,"[saw, descent, night, stockholm, film, festiv,..."
4998,99_1.txt,"b""Some films that you pick up for a pound turn...",0,"[film, pick, pound, turn, good, 23rd, centuri,..."


In [16]:
tokens = casual_tokenize(new_train, reduce_len=True, strip_handles=True)

TypeError: expected string or buffer

In [ ]:
tokens=tokenize(new_train.readline)

In [ ]:
def normalize_document(doc):
    # lower

In [ ]:
def get_data(split):
    if split.lower() == 'train':
        folder = 'train'
    elif split.lower() == 'test':
        folder = 'test'
    else:
        raise ValueError('Invalid data split specified.')
        
    file_names = []
    text = []
    is_positive = []
    
    # read all positive files
    files = glob.glob(os.path.join("data", folder, 'pos', '*'))
    for file in files:
        head, tail = os.path.split(file)
        file_names.append(tail)
        is_positive.append(1)
        with open(file, 'rb') as open_file:
            text.append(open_file.readlines()[0])
            
    # read all negative files
    files = glob.glob(os.path.join("data", folder, 'neg', '*'))
    for file in files:
        head, tail = os.path.split(file)
        file_names.append(tail)
        is_positive.append(0)
        with open(file, 'rb') as open_file:
            text.append(open_file.readlines()[0])
            
    return pd.DataFrame(data={'file': file_names, 'text': text, 'is_positive': is_positive})


train_df = get_data('train')
train_df['text'] = train_df['text'].astype(str)
train_df